#### Import Library

In [ ]:
# library
import torch
import torch.nn.functional as F
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torchvision.models as models
from torch.optim import SGD
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

# parameters
torch.manual_seed(1)
batch_size = 20

#### Read data from images and excel-labels

In [ ]:
# read the images and names
IMAGE_DIR =  'Data/C_img_jpg'
def read_images(image_path=IMAGE_DIR):
    images = []
    images_names = [image for image in os.listdir(image_path) if not image.startswith('.')] 
    for image_name in images_names: 
            img = Image.open (os.path.join(image_path, image_name))
            images.append(img)
    return images,images_names
images,names = read_images()

In [ ]:
# read the label
excel=pd.read_csv('Data/C_all_data_add_20211007_xz.csv', encoding = 'gb2312')
excel = np.array(excel)
excel = excel.tolist()
# Alligning the labels with images. (i.e. the first label corresponds to the first label.)
labels = []
sub_labels = []

for i in range(len(names)):
    for j in range(len(excel)):
        if names[i] == excel[j][0]:
            labels.append(excel[j][1])
            sub_labels.append(excel[j][2:])


#### Split the training and testing dataet

In [ ]:
# Stratified Random Sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)

names = np.array(names)
labels = np.array(labels)
names = names.reshape((names.shape[0],-1))
labels = labels.reshape((labels.shape[0],-1))
data = np.hstack((names,labels)) # hstack:each name corresponds to one label 
for train_index,test_index in split.split(data,data[:,-1]):
    train_set = data[train_index,:]
    test_set = data[test_index,:]

#### Data Preprocessing

In [ ]:
# data trsnformation
train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(0.5),
                                       transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       
                                       ])
train_data_1 = []
train_data_3 = []
for row in range(len(train_set)):
    for i in range(len(names)):
        if names[i] == train_set[row][0]: 
            img_data = train_transforms(images[i]) 
            # train_set is [names, labels], we need to get the train_data_1 [images, labels], train_data_3 [images, sub_labels, labels]
            train_data_1.append((img_data,int(train_set[row][1]))) 
            train_data_3.append((img_data,torch.Tensor(sub_labels[i][:]),int(train_set[row][1])))

test_transforms =  transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       
                                       ])
test_data_1 = []
test_data_3 = []
for row in range(len(test_set)):
    for i in range(len(names)):
        if names[i] == test_set[row][0]:
            img_data = test_transforms(images[i])
            test_data_1.append((img_data,int(test_set[row][1]))) 
            test_data_3.append((img_data,torch.Tensor(sub_labels[i][:]),int(train_set[row][1])))

In [ ]:
# Dataloader
test_size = 0.2
train_loader_1 = DataLoader(train_data_1 , batch_size = batch_size ,shuffle=True)
test_loader_1 = DataLoader(test_data_1, batch_size = int(batch_size*test_size/(1-test_size)), shuffle=False) #int(batch_size*test_size/(1-test_size))

#### Load the Network

In [ ]:
# model-1: resnet18 
device=torch.device('cuda')
model_1 = models.resnet18(pretrained=True)

# modify the last layer
model_1.fc =  nn.Linear(model_1.fc.in_features, 7)
model_1 = model_1.to(device)
print(model_1)

#### Train the Network

In [ ]:
# mixup_data
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
# model 1 training
learning_rate = 0.005
num_epoches = 20
num_classes = 7
#alpha = 0.2
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model_1.parameters(),lr=learning_rate)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)     # learning rate decay

train_loss_all = []   # store the loss of training set
train_accur_all = []  # store the accuracy of training set
test_loss_all = []    # store the loss of test set
test_accur_all = []   # store the accuracy of test set

# start training
for epoch in range(num_epoches):
    running_loss=0.0
    running_acc=0.0
    model_1.train()
    for i,data in enumerate(train_loader_1,1):
        img,label=data
        img=img.cuda()
        label=label.cuda()
        #data, labels_a, labels_b, lam = mixup_data(img, label, alpha)
        img=Variable(img)
        label=Variable(label)
        
        # forward
        out=model_1(img)
        loss=criterion(out,label)
        #loss = mixup_criterion(criterion, out, labels_a, labels_b, lam)
        running_loss+=loss.data*label.size(0)
        _,pred=torch.max(out,1)
        num_correct=(pred==label).sum()
        running_acc+=num_correct.data
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #scheduler.step() # update learning rate
    
    print('Train {} epoch, Loss: {:.6f},Acc: {:.6f}'.format(epoch+1,running_loss/(len(train_data_1)),running_acc/len(train_data_1)))
    train_loss_all.append(running_loss / len(train_data_1))   # store the loss of training set, and then plot it
    train_accur_all.append(running_acc/len(train_data_1))     # store the accuracy of training set, and then plot it
    
    model_1.eval()
    with torch.no_grad():
        eval_loss = 0
        eval_acc = 0
        train_loss_results = []

        for data in test_loader_1:
            img, label = data
            img = Variable(img)
            img=img.cuda()
            label=label.cuda()
            out = model_1(img)
            loss = criterion(out, label)
            eval_loss += loss.data
            _,pred = torch.max(out,1)
            num_correct = (pred == label).sum()
            eval_acc += num_correct.data

    print('Test Loss: {:,.6f}, Acc: {:,.6f}'.format(eval_loss/(len(test_data_1)), eval_acc/(len(test_data_1))))
    test_loss_all.append(eval_loss/(len(test_data_1)))
    test_accur_all.append(eval_acc/(len(test_data_1)))

#### Result Visualization

In [ ]:
# transfer the data to cpu
for i in range(len(train_loss_all)):
    train_loss_all[i] = torch.Tensor.cpu(train_loss_all[i])
    train_accur_all[i] = torch.Tensor.cpu(train_accur_all[i])
    test_loss_all[i] = torch.Tensor.cpu(test_loss_all[i])
    test_accur_all[i] = torch.Tensor.cpu(test_accur_all[i])

epoches = [i+1 for i in range(num_epoches)]

# plot the loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(2, 2, 1)
plt.plot(epoches, train_loss_all,"ro-", label="Train loss")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
plt.subplot(2, 2, 2)
plt.plot(epoches, train_accur_all,"ro-", label="Train accur")
plt.xlabel("epoch")
plt.ylabel("acc")
plt.legend()
plt.subplot(2, 2, 3)
plt.plot(epoches, test_loss_all,"bo-", label="Test loss")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
plt.subplot(2, 2, 4)
plt.plot(epoches, test_accur_all,"bo-", label="Test accur")
plt.xlabel("epoch")
plt.ylabel("acc")
plt.legend()
plt.show()

#### Save the model


In [ ]:
# save the model
torch.save(model_1, 'model_1_new.pth')